# OCRで抽出した矩形領域を並び替えるロジック

## クラスタ１

* まず同じ行のものを見つける
    * 同じ行はymin-ymaxにDuplicateがあるもの
* 左端から見ていって距離が近いものはまとめる
    * 近さは高さのMedianから、文字1個分を定義し、それの何個分かで測る。とりあえず１０文字にする


あとは階層型クラスタリングしてみるか

In [1]:
import json
from collections import namedtuple

In [2]:
j = '{"uuid":"","result":{"boxes":[{"adjusted_rect":{"xmax":662,"xmin":70,"ymax":634,"ymin":613},"label":null,"raw_text":"尚、お支払期日が土日祝祭日の場合は前営業日までにお支払い下さい。","rect":{"xmax":662,"xmin":70,"ymax":634,"ymin":613},"text":"尚、お支払期日が土日祝祭日の場合は前営業日までにお支払い下さい。"},{"adjusted_rect":{"xmax":1452,"xmin":1310,"ymax":374,"ymin":331},"label":null,"raw_text":"文","rect":{"xmax":1452,"xmin":1310,"ymax":374,"ymin":331},"text":"文"},{"adjusted_rect":{"xmax":1049,"xmin":916,"ymax":635,"ymin":607},"label":null,"raw_text":"F006\u003cUNK\u003e779","rect":{"xmax":1049,"xmin":916,"ymax":635,"ymin":607},"text":"F006\u003cUNK\u003e779"},{"adjusted_rect":{"xmax":286,"xmin":107,"ymax":349,"ymin":324},"label":null,"raw_text":"新規事業推進室","rect":{"xmax":286,"xmin":107,"ymax":349,"ymin":324},"text":"新規事業推進室"},{"adjusted_rect":{"xmax":1217,"xmin":1022,"ymax":345,"ymin":322},"label":null,"raw_text":"日比谷国際ビル1階","rect":{"xmax":1217,"xmin":1022,"ymax":345,"ymin":322},"text":"日比谷国際ビル1階"},{"adjusted_rect":{"xmax":1266,"xmin":1151,"ymax":634,"ymin":607},"label":null,"raw_text":"F758(Tt","rect":{"xmax":1266,"xmin":1151,"ymax":634,"ymin":607},"text":"F758(Tt"},{"adjusted_rect":{"xmax":1479,"xmin":1353,"ymax":631,"ymin":606},"label":null,"raw_text":"F75/+99","rect":{"xmax":1479,"xmin":1353,"ymax":631,"ymin":606},"text":"F75/+99"},{"adjusted_rect":{"xmax":1249,"xmin":1021,"ymax":390,"ymin":366},"label":null,"raw_text":"株式会社ユニメディア","rect":{"xmax":1249,"xmin":1021,"ymax":390,"ymin":366},"text":"株式会社ユニメディア"},{"adjusted_rect":{"xmax":314,"xmin":103,"ymax":395,"ymin":369},"label":null,"raw_text":"逆瀨川光人様","rect":{"xmax":314,"xmin":103,"ymax":395,"ymin":369},"text":"逆瀨川光人様"},{"adjusted_rect":{"xmax":289,"xmin":96,"ymax":258,"ymin":234},"label":null,"raw_text":"MG白金台ビル4F","rect":{"xmax":289,"xmin":96,"ymax":258,"ymin":234},"text":"MG白金台ビル4F"},{"adjusted_rect":{"xmax":1041,"xmin":851,"ymax":690,"ymin":669},"label":null,"raw_text":"薮量単位","rect":{"xmax":1041,"xmin":851,"ymax":690,"ymin":669},"text":"薮量単位"},{"adjusted_rect":{"xmax":413,"xmin":103,"ymax":304,"ymin":279},"label":null,"raw_text":"ウォンテッドリー株式会社","rect":{"xmax":413,"xmin":103,"ymax":304,"ymin":279},"text":"ウォンテッドリー株式会社"},{"adjusted_rect":{"xmax":1424,"xmin":1339,"ymax":688,"ymin":667},"label":null,"raw_text":"金額","rect":{"xmax":1424,"xmin":1339,"ymax":688,"ymin":667},"text":"金額"},{"adjusted_rect":{"xmax":1292,"xmin":1022,"ymax":300,"ymin":277},"label":null,"raw_text":"東京都千代田区内幸町2-2","rect":{"xmax":1292,"xmin":1022,"ymax":300,"ymin":277},"text":"東京都千代田区内幸町2-2"},{"adjusted_rect":{"xmax":1150,"xmin":1020,"ymax":254,"ymin":234},"label":null,"raw_text":"〒100-0011","rect":{"xmax":1150,"xmin":1020,"ymax":254,"ymin":234},"text":"〒100-0011"},{"adjusted_rect":{"xmax":1266,"xmin":1207,"ymax":733,"ymin":706},"label":null,"raw_text":"25円","rect":{"xmax":1266,"xmin":1207,"ymax":733,"ymin":706},"text":"25円"},{"adjusted_rect":{"xmax":490,"xmin":230,"ymax":596,"ymin":572},"label":null,"raw_text":"2019年01月31日","rect":{"xmax":490,"xmin":230,"ymax":596,"ymin":572},"text":"2019年01月31日"},{"adjusted_rect":{"xmax":214,"xmin":82,"ymax":597,"ymin":574},"label":null,"raw_text":"お支払期日","rect":{"xmax":214,"xmin":82,"ymax":597,"ymin":574},"text":"お支払期日"},{"adjusted_rect":{"xmax":1453,"xmin":1313,"ymax":590,"ymin":568},"label":null,"raw_text":"回御求額","rect":{"xmax":1453,"xmin":1313,"ymax":590,"ymin":568},"text":"回御求額"},{"adjusted_rect":{"xmax":639,"xmin":92,"ymax":445,"ymin":420},"label":null,"raw_text":"平素は格別のお引き立てを賜り厚く御礼申し上げます。","rect":{"xmax":639,"xmin":92,"ymax":445,"ymin":420},"text":"平素は格別のお引き立てを賜り厚く御礼申し上げます。"},{"adjusted_rect":{"xmax":1225,"xmin":1097,"ymax":591,"ymin":569},"label":null,"raw_text":"消賀税額等","rect":{"xmax":1225,"xmin":1097,"ymax":591,"ymin":569},"text":"消賀税額等"},{"adjusted_rect":{"xmax":1016,"xmin":862,"ymax":592,"ymin":569},"label":null,"raw_text":"抜御買上額","rect":{"xmax":1016,"xmin":862,"ymax":592,"ymin":569},"text":"抜御買上額"},{"adjusted_rect":{"xmax":405,"xmin":83,"ymax":485,"ymin":462},"label":null,"raw_text":"下記の通り御請求申し上げます","rect":{"xmax":405,"xmin":83,"ymax":485,"ymin":462},"text":"下記の通り御請求申し上げます"},{"adjusted_rect":{"xmax":1474,"xmin":1274,"ymax":434,"ymin":414},"label":null,"raw_text":"AX:03-5511-2782","rect":{"xmax":1474,"xmin":1274,"ymax":434,"ymin":414},"text":"AX:03-5511-2782"},{"adjusted_rect":{"xmax":1234,"xmin":1022,"ymax":436,"ymin":416},"label":null,"raw_text":"TEL:03-5511-2781","rect":{"xmax":1234,"xmin":1022,"ymax":436,"ymin":416},"text":"TEL:03-5511-2781"},{"adjusted_rect":{"xmax":483,"xmin":86,"ymax":787,"ymin":762},"label":null,"raw_text":"名刺入力代行(API連携)/unit2018407","rect":{"xmax":483,"xmin":86,"ymax":787,"ymin":762},"text":"名刺入力代行(API連携)/unit2018407"},{"adjusted_rect":{"xmax":239,"xmin":87,"ymax":117,"ymin":81},"label":null,"raw_text":"請求晝","rect":{"xmax":239,"xmin":87,"ymax":117,"ymin":81},"text":"請求晝"},{"adjusted_rect":{"xmax":489,"xmin":82,"ymax":740,"ymin":714},"label":null,"raw_text":"名刺入力代行(API連携)/unit20184079","rect":{"xmax":489,"xmin":82,"ymax":740,"ymin":714},"text":"名刺入力代行(API連携)/unit20184079"},{"adjusted_rect":{"xmax":1478,"xmin":1211,"ymax":105,"ymin":82},"label":null,"raw_text":"発行日:2019年01月07日","rect":{"xmax":1478,"xmin":1211,"ymax":105,"ymin":82},"text":"発行日:2019年01月07日"},{"adjusted_rect":{"xmax":451,"xmin":89,"ymax":115,"ymin":81},"label":null,"raw_text":"御請求書2018年12月度","rect":{"xmax":451,"xmin":89,"ymax":115,"ymin":81},"text":"御請求書2018年12月度"},{"adjusted_rect":{"xmax":222,"xmin":87,"ymax":168,"ymin":143},"label":null,"raw_text":"〒108-00731","rect":{"xmax":222,"xmin":87,"ymax":168,"ymin":143},"text":"〒108-00731"},{"adjusted_rect":{"xmax":451,"xmin":273,"ymax":112,"ymin":87},"label":null,"raw_text":"2018年12月度","rect":{"xmax":451,"xmin":273,"ymax":112,"ymin":87},"text":"2018年12月度"},{"adjusted_rect":{"xmax":1477,"xmin":1352,"ymax":733,"ymin":708},"label":null,"raw_text":"82-900F","rect":{"xmax":1477,"xmin":1352,"ymax":733,"ymin":708},"text":"82-900F"},{"adjusted_rect":{"xmax":1477,"xmin":1261,"ymax":161,"ymin":139},"label":null,"raw_text":"請求No:000007355","rect":{"xmax":1477,"xmin":1261,"ymax":161,"ymin":139},"text":"請求No:000007355"},{"adjusted_rect":{"xmax":1474,"xmin":1366,"ymax":782,"ymin":756},"label":null,"raw_text":"t000\u003cUNK\u003e0t","rect":{"xmax":1474,"xmin":1366,"ymax":782,"ymin":756},"text":"t000\u003cUNK\u003e0t"},{"adjusted_rect":{"xmax":1013,"xmin":840,"ymax":739,"ymin":710},"label":null,"raw_text":"x9TS611","rect":{"xmax":1013,"xmin":840,"ymax":739,"ymin":710},"text":"x9TS611"},{"adjusted_rect":{"xmax":1260,"xmin":1149,"ymax":783,"ymin":758},"label":null,"raw_text":"F-0000t","rect":{"xmax":1260,"xmin":1149,"ymax":783,"ymin":758},"text":"F-0000t"},{"adjusted_rect":{"xmax":1482,"xmin":1404,"ymax":66,"ymin":46},"label":null,"raw_text":"i/1page","rect":{"xmax":1482,"xmin":1404,"ymax":66,"ymin":46},"text":"i/1page"},{"adjusted_rect":{"xmax":376,"xmin":92,"ymax":214,"ymin":189},"label":null,"raw_text":"東京都港区白金台5-12-7","rect":{"xmax":376,"xmin":92,"ymax":214,"ymin":189},"text":"東京都港区白金台5-12-7"},{"adjusted_rect":{"xmax":1268,"xmin":1018,"ymax":212,"ymin":173},"label":null,"raw_text":"MUNIMEDIA","rect":{"xmax":1268,"xmin":1018,"ymax":212,"ymin":173},"text":"MUNIMEDIA"},{"adjusted_rect":{"xmax":1205,"xmin":1132,"ymax":689,"ymin":669},"label":null,"raw_text":"価","rect":{"xmax":1205,"xmin":1132,"ymax":689,"ymin":669},"text":"価"}],"component_params":{"recognizer":{"model":"general"}}}}'

In [3]:
def _json_object_hook(d): return namedtuple('X', d.keys())(*d.values())
def json2obj(data): return json.loads(data, object_hook=_json_object_hook)

x = json2obj(j)

In [4]:
boxes = x.result.boxes

In [5]:
import heapq
import numpy as np

def _rect_distance(lhs, rhs):
    hdiff = 0 if lhs.ymax <= rhs.ymin and lhs.ymin >= rhs.ymax else min(abs(lhs.ymax-rhs.ymin), abs(lhs.ymin-rhs.ymax))
    wdiff = 0 if lhs.xmax <= rhs.xmin and lhs.xmin >= rhs.xmax else min(abs(lhs.xmax-rhs.xmin), abs(lhs.xmin-rhs.xmax))
    if hdiff == 0 and wdiff <= 30:
        wdiff /= 5
    return hdiff * 2 + wdiff

class BoxClusterer:
    def __init__(self, boxes):
        self.boxes = boxes
        self._make_cluster([b.rect for b in boxes])

    def _make_cluster(self, rects):
        n = len(rects)
        self.heap = []
        for i in range(n):
            for j in range(i+1, n):
                self.heap.append((_rect_distance(rects[i], rects[j]), i, j))
        heapq.heapify(self.heap)

        cuf = ClusterUnionFind(rects)
        while cuf.size > 1:
            dist, i, j = heapq.heappop(self.heap)
            print(dist, boxes[i].raw_text, boxes[j].raw_text)
            cuf.union(i, j)
        self.root = cuf.clusters[cuf.find(0)]

    def sorted(self):
        self._traversed = []
        self._traverse(self.root)
        return [self.boxes[i] for i in self._traversed]

    def _traverse(self, node):
        if node.index is not None:
            self._traversed.append(node.index)
            return
        self._traverse(node.left)
        self._traverse(node.right)

class BoxCluster(object):
    def __init__(self, pos, left, right, index):
        self.pos = pos
        self.left = left
        self.right = right
        self.index = index

class BoxClusterLeaf(BoxCluster):
    def __init__(self, index, rect):
        super().__init__((rect.ymin, rect.xmin), None, None, index)

class BoxClusterTree(BoxCluster):
    def __init__(self, lhs, rhs):
        if lhs.pos > rhs.pos:
            lhs, rhs = rhs, lhs
        super().__init__(lhs.pos, lhs, rhs, None)

class ClusterUnionFind:
    def __init__(self, rects):
        self.size = len(rects)
        self.par = [i for i in range(self.size)]
        self.rank = [0] * self.size
        self.clusters = [BoxClusterLeaf(i, rects[i]) for i in range(self.size)]

    def find(self, x):
        if self.par[x] == x:
            return x
        else:
            self.par[x] = self.find(self.par[x])
            return self.par[x]

    def union(self, x, y):
        x = self.find(x)
        y = self.find(y)
        if x == y: return
        self.size -= 1
        cluster = BoxClusterTree(self.clusters[x], self.clusters[y])
        if self.rank[x] < self.rank[y]:
            self.par[x] = y
            self.clusters[y] = cluster
        else:
            self.par[y] = x
            self.clusters[x] = cluster
            if self.rank[x] == self.rank[y]:
                self.rank[x] += 1

bc = BoxClusterer(boxes)
sorted_boxes = bc.sorted()

31 請求No:000007355 MUNIMEDIA
36 25円 価
60 2019年01月31日 お支払期日
73 株式会社ユニメディア AX:03-5511-2782
76 AX:03-5511-2782 TEL:03-5511-2781
77 文 株式会社ユニメディア
80 文 東京都千代田区内幸町2-2
80 F006<UNK>779 消賀税額等
81 F758(Tt 回御求額
94 請求晝 2018年12月度
103 25円 F-0000t
106 F758(Tt 消賀税額等
106 発行日:2019年01月07日 i/1page
109 金額 25円
112 金額 82-900F
113 〒108-00731 2018年12月度
121 文 日比谷国際ビル1階
124 F758(Tt 価
125 消賀税額等 抜御買上額
130 F006<UNK>779 抜御買上額
130 回御求額 消賀税額等
131 薮量単位 価
132 F75/+99 回御求額
135 F758(Tt F75/+99
136 25円 82-900F
139 F758(Tt 金額
142 82-900F F-0000t
143 F75/+99 金額
146 25円 t000<UNK>0t
151 F006<UNK>779 価
154 t000<UNK>0t F-0000t
156 F006<UNK>779 F758(Tt
157 82-900F t000<UNK>0t
158 F75/+99 消賀税額等
160 文 TEL:03-5511-2781
161 x9TS611 価
165 F758(Tt 抜御買上額
172 金額 価
172 〒108-00731 東京都港区白金台5-12-7
174 東京都千代田区内幸町2-2 〒100-0011
174 x9TS611 F-0000t
176 尚、お支払期日が土日祝祭日の場合は前営業日までにお支払い下さい。 お支払期日
176 〒100-0011 MUNIMEDIA
180 F758(Tt 薮量単位
185 82-900F 価
187 請求晝 〒108-00731
189 御請求書2018年12月度 〒108-00731
193 F006<UNK>779 薮量単位
193 発行日:2019年01月07日 MUNIMEDIA
194 金

In [6]:
text = ""
for box in boxes:
    text += box.raw_text + "\n"
print(text)

尚、お支払期日が土日祝祭日の場合は前営業日までにお支払い下さい。
文
F006<UNK>779
新規事業推進室
日比谷国際ビル1階
F758(Tt
F75/+99
株式会社ユニメディア
逆瀨川光人様
MG白金台ビル4F
薮量単位
ウォンテッドリー株式会社
金額
東京都千代田区内幸町2-2
〒100-0011
25円
2019年01月31日
お支払期日
回御求額
平素は格別のお引き立てを賜り厚く御礼申し上げます。
消賀税額等
抜御買上額
下記の通り御請求申し上げます
AX:03-5511-2782
TEL:03-5511-2781
名刺入力代行(API連携)/unit2018407
請求晝
名刺入力代行(API連携)/unit20184079
発行日:2019年01月07日
御請求書2018年12月度
〒108-00731
2018年12月度
82-900F
請求No:000007355
t000<UNK>0t
x9TS611
F-0000t
i/1page
東京都港区白金台5-12-7
MUNIMEDIA
価



In [7]:
text = ""
for box in sorted_boxes:
    text += box.raw_text + "\n"
print(text)

i/1page
発行日:2019年01月07日
請求No:000007355
MUNIMEDIA
〒100-0011
東京都千代田区内幸町2-2
文
株式会社ユニメディア
AX:03-5511-2782
TEL:03-5511-2781
日比谷国際ビル1階
回御求額
F758(Tt
消賀税額等
F006<UNK>779
金額
価
25円
F-0000t
82-900F
抜御買上額
薮量単位
F75/+99
t000<UNK>0t
x9TS611
2019年01月31日
お支払期日
尚、お支払期日が土日祝祭日の場合は前営業日までにお支払い下さい。
下記の通り御請求申し上げます
請求晝
2018年12月度
〒108-00731
東京都港区白金台5-12-7
御請求書2018年12月度
MG白金台ビル4F
ウォンテッドリー株式会社
新規事業推進室
逆瀨川光人様
平素は格別のお引き立てを賜り厚く御礼申し上げます。
名刺入力代行(API連携)/unit20184079
名刺入力代行(API連携)/unit2018407

